# Тестовое задание logiclike

Техническая часть.
В приложенном файле содержатся логи действий пользователей сайта в хронологическом порядке. 
При просмотре любой страницы сайта счетчик передает событие, хранящее от двух до четырех свойств пользователя:
- Уникальный id пользователя в системе аналитики (колонка clientID, обязательное свойство)
- Url просмотренной страницы (колонка URL, обязательное свойство);
- Выполнение одной из доступных конверсионных/ретаргетинговых целей (колонка goalsID);
- Личная информация пользователя, представленная в виде пар ключ/значение (колонки paramsKeys и paramsValues).

Пользователь может либо просматривать "информационный" контент сайта, либо решать логические задачи. Url страниц с логическими задачами содержат ключевое слово "quiz" и ее уникальный номер. Url просмотренной задачи содержит ключевое слово "process", решенной - "result". Остальные строки содержат url-заглушку.

Доступные ID целей:
- 25230759: Первый визит пользователя
- 40343059: Пользователь смотрел тарифы

Требуется разбить пользователей на три возрастные когорты, используя в качестве фильтра параметр targetLessons колонки paramsKeys.
В рамках каждой когорты пользователей необходимо разбить на две группы: совершившие цель "Первый визит" (новые пользователи сайта) и не совершившие ее (повторные визиты).
Для группы рассчитать следующие показатели:
- Количество пользователей в группе;
- Процент пользователей, достигших цели "Смотрел тарифы";
- Среднее количество уникальных решенных задач на активного пользователя. Активными считаем только тех пользователей, которые просмотрели хотя бы одну задачу. 

Решение требуется предоставить в виде таблицы и исходного кода в среде python либо jupyter notebook.

Аналитическая часть.
На основании полученных данных сделать вывод о том, какая возрастная когорта пользователей является наиболее выгодной для привлечения. Предполагается, что конверсия из просмотра страницы тарифов в покупку является константой.

## Шаг 1. Предобработка данных

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [2]:
df = pd.read_csv('data.csv') # чтение и сохранение файла

In [3]:
df.columns = df.columns.str.lower() # приведение названия столбцов к нижнему регистру

In [4]:
df.info() # общая информация о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306383 entries, 0 to 306382
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   unnamed: 0    306383 non-null  int64  
 1   clientid      306383 non-null  uint64 
 2   url           306383 non-null  object 
 3   goalsid       6627 non-null    float64
 4   paramskeys    306383 non-null  object 
 5   paramsvalues  306383 non-null  object 
dtypes: float64(1), int64(1), object(3), uint64(1)
memory usage: 14.0+ MB


In [5]:
df.duplicated().sum() # проверка на дубликаты

0

In [6]:
df.paramskeys.unique() # уникальные значения столбца paramskeys

array(["['cardId','region','userId','registrationType','targetLessons']",
       "['ip']", '[]', "['targetLessons']",
       "['targetLessons','cardId','userId','registrationType']",
       "['cardId','userId','registrationType']"], dtype=object)

In [7]:
df.url.value_counts() # уникальные значения столбца url

https://logiclike.com/                               21231
https://logiclike.com/cabinet#/quiz/73376/result      3296
https://logiclike.com/cabinet#/quiz/73429/result      2969
https://logiclike.com/cabinet#/quiz/75472/result      2881
https://logiclike.com/cabinet#/quiz/75473/result      2773
                                                     ...  
https://logiclike.com/cabinet#/quiz/74246/process        1
https://logiclike.com/cabinet#/quiz/74861/result         1
https://logiclike.com/user#/quiz/73523/process           1
https://logiclike.com/cabinet#/quiz/71923/process        1
https://logiclike.com/cabinet#/quiz/73458/result         1
Name: url, Length: 6100, dtype: int64

In [8]:
df.clientid.value_counts() # уникальные значения столбца url

15471369431036177185    859
1547135229406530091     770
1542904295784898289     677
1547117842997870685     670
1544447854232165163     631
                       ... 
154440341493575222        1
1546612274550244282       1
1514576748566860735       1
1547217644903173780       1
1547204090513626392       1
Name: clientid, Length: 8490, dtype: int64

In [9]:
df

,unnamed: 0,clientid,url,goalsid,paramskeys,paramsvalues
0,0,1543250134295194505,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['215499','Россия','221529','null','2']"
1,1,1540396567213978260,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['394451','Минск','340628','null','0']"
2,2,1540396567213978260,https://logiclike.com/,NaN,['ip'],['178.120.38.195']
3,3,1540396567213978260,https://logiclike.com/user#/quiz/73985/process,NaN,[],[]
4,4,1540396567213978260,https://logiclike.com/cabinet#/quiz/73985/process,NaN,['ip'],['178.120.38.195']
...,...,...,...,...,...,...
306378,306378,1547201659806207786,https://logiclike.com/cabinet#/quiz/73429/result,NaN,"['cardId','region','userId','registrationType'...","['877460','Россия','880366','18','2']"
306379,306379,1547211249801412828,https://logiclike.com/,NaN,['ip'],['85.141.127.134']
306380,306380,1547211249801412828,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['878249','Россия','881170','18','2']"
306381,306381,1547211249801412828,https://logiclike.com/,25230759.0,[],[]


#### Добавим новый столбец с возрастной когортой, используя в качестве фильтра параметр targetLessons колонки paramsKeys и paramsvalues.

In [10]:
df['targetlessons'] = df[['paramskeys', 'paramsvalues']].apply(lambda x: int(list(x['paramsvalues'])[-3]) if x['paramskeys'] == "['cardId','region','userId','registrationType','targetLessons']" or x['paramskeys'] == "['targetLessons']"
                                                                                    else int(list(x['paramsvalues'])[2]) if x['paramskeys'] == "['targetLessons','cardId','userId','registrationType']"
                                                                                    else np.nan, axis=1)

In [11]:
df

,unnamed: 0,clientid,url,goalsid,paramskeys,paramsvalues,targetlessons
0,0,1543250134295194505,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['215499','Россия','221529','null','2']",2.0
1,1,1540396567213978260,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['394451','Минск','340628','null','0']",0.0
2,2,1540396567213978260,https://logiclike.com/,NaN,['ip'],['178.120.38.195'],NaN
3,3,1540396567213978260,https://logiclike.com/user#/quiz/73985/process,NaN,[],[],NaN
4,4,1540396567213978260,https://logiclike.com/cabinet#/quiz/73985/process,NaN,['ip'],['178.120.38.195'],NaN
...,...,...,...,...,...,...,...
306378,306378,1547201659806207786,https://logiclike.com/cabinet#/quiz/73429/result,NaN,"['cardId','region','userId','registrationType'...","['877460','Россия','880366','18','2']",2.0
306379,306379,1547211249801412828,https://logiclike.com/,NaN,['ip'],['85.141.127.134'],NaN
306380,306380,1547211249801412828,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['878249','Россия','881170','18','2']",2.0
306381,306381,1547211249801412828,https://logiclike.com/,25230759.0,[],[],NaN


#### Добавим новый столбец с отметкой о просмотре тарифов (1 - смотрел тарифы).

In [12]:
df['tarifs'] = df['goalsid'].apply(lambda x: 1 if x == 40343059 else 0)

In [13]:
df

,unnamed: 0,clientid,url,goalsid,paramskeys,paramsvalues,targetlessons,tarifs
0,0,1543250134295194505,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['215499','Россия','221529','null','2']",2.0,0
1,1,1540396567213978260,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['394451','Минск','340628','null','0']",0.0,0
2,2,1540396567213978260,https://logiclike.com/,NaN,['ip'],['178.120.38.195'],NaN,0
3,3,1540396567213978260,https://logiclike.com/user#/quiz/73985/process,NaN,[],[],NaN,0
4,4,1540396567213978260,https://logiclike.com/cabinet#/quiz/73985/process,NaN,['ip'],['178.120.38.195'],NaN,0
...,...,...,...,...,...,...,...,...
306378,306378,1547201659806207786,https://logiclike.com/cabinet#/quiz/73429/result,NaN,"['cardId','region','userId','registrationType'...","['877460','Россия','880366','18','2']",2.0,0
306379,306379,1547211249801412828,https://logiclike.com/,NaN,['ip'],['85.141.127.134'],NaN,0
306380,306380,1547211249801412828,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['878249','Россия','881170','18','2']",2.0,0
306381,306381,1547211249801412828,https://logiclike.com/,25230759.0,[],[],NaN,0


#### Добавим новый столбец с отметкой о решении задачи (1 - решена, 0 - в процессе).

In [14]:
df['quiz_status'] = df['url'].apply(lambda x: 1 if 'result' in x else 0 if 'process' in x else np.nan)

In [15]:
df

,unnamed: 0,clientid,url,goalsid,paramskeys,paramsvalues,targetlessons,tarifs,quiz_status
0,0,1543250134295194505,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['215499','Россия','221529','null','2']",2.0,0,NaN
1,1,1540396567213978260,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['394451','Минск','340628','null','0']",0.0,0,NaN
2,2,1540396567213978260,https://logiclike.com/,NaN,['ip'],['178.120.38.195'],NaN,0,NaN
3,3,1540396567213978260,https://logiclike.com/user#/quiz/73985/process,NaN,[],[],NaN,0,0.0
4,4,1540396567213978260,https://logiclike.com/cabinet#/quiz/73985/process,NaN,['ip'],['178.120.38.195'],NaN,0,0.0
...,...,...,...,...,...,...,...,...,...
306378,306378,1547201659806207786,https://logiclike.com/cabinet#/quiz/73429/result,NaN,"['cardId','region','userId','registrationType'...","['877460','Россия','880366','18','2']",2.0,0,1.0
306379,306379,1547211249801412828,https://logiclike.com/,NaN,['ip'],['85.141.127.134'],NaN,0,NaN
306380,306380,1547211249801412828,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['878249','Россия','881170','18','2']",2.0,0,NaN
306381,306381,1547211249801412828,https://logiclike.com/,25230759.0,[],[],NaN,0,NaN


#### Добавим новый столбец с отметкой о первом и повторном визите (0 - новый пользователь, 4 - повторный визит).

In [16]:
df['new_client'] = df['goalsid'].apply(lambda x: 0 if x == 25230759 else 4)

In [17]:
df

,unnamed: 0,clientid,url,goalsid,paramskeys,paramsvalues,targetlessons,tarifs,quiz_status,new_client
0,0,1543250134295194505,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['215499','Россия','221529','null','2']",2.0,0,NaN,4
1,1,1540396567213978260,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['394451','Минск','340628','null','0']",0.0,0,NaN,4
2,2,1540396567213978260,https://logiclike.com/,NaN,['ip'],['178.120.38.195'],NaN,0,NaN,4
3,3,1540396567213978260,https://logiclike.com/user#/quiz/73985/process,NaN,[],[],NaN,0,0.0,4
4,4,1540396567213978260,https://logiclike.com/cabinet#/quiz/73985/process,NaN,['ip'],['178.120.38.195'],NaN,0,0.0,4
...,...,...,...,...,...,...,...,...,...,...
306378,306378,1547201659806207786,https://logiclike.com/cabinet#/quiz/73429/result,NaN,"['cardId','region','userId','registrationType'...","['877460','Россия','880366','18','2']",2.0,0,1.0,4
306379,306379,1547211249801412828,https://logiclike.com/,NaN,['ip'],['85.141.127.134'],NaN,0,NaN,4
306380,306380,1547211249801412828,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['878249','Россия','881170','18','2']",2.0,0,NaN,4
306381,306381,1547211249801412828,https://logiclike.com/,25230759.0,[],[],NaN,0,NaN,0


#### Для правильного подсчета количества решенных задач и задач в процессе удалим дубликаты (будем считать, что у одной задачи для каждого клиента возможно только по одному варианту process и result.

Добавим новый столбец с номером задачи (т.к. в url могут быть разные ссылки на одну задачу).

In [18]:
df['quiz_number'] = df['url'].apply(lambda x: re.sub('\D+', "", x)) # извлечение только цифр из url

In [19]:
df = df.drop_duplicates(subset=['clientid', 'goalsid', 'quiz_status', 'quiz_number']) # удаление дубликатов

In [20]:
df

,unnamed: 0,clientid,url,goalsid,paramskeys,paramsvalues,targetlessons,tarifs,quiz_status,new_client,quiz_number
0,0,1543250134295194505,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['215499','Россия','221529','null','2']",2.0,0,NaN,4,
1,1,1540396567213978260,https://logiclike.com/,NaN,"['cardId','region','userId','registrationType'...","['394451','Минск','340628','null','0']",0.0,0,NaN,4,
3,3,1540396567213978260,https://logiclike.com/user#/quiz/73985/process,NaN,[],[],NaN,0,0.0,4,73985
7,7,1540396567213978260,https://logiclike.com/cabinet#/quiz/73985/result,NaN,[],[],NaN,0,1.0,4,73985
8,8,1540396567213978260,https://logiclike.com/cabinet#/quiz/73987/process,NaN,[],[],NaN,0,0.0,4,73987
...,...,...,...,...,...,...,...,...,...,...,...
306376,306376,1547201659806207786,https://logiclike.com/cabinet#/quiz/73429/process,NaN,[],[],NaN,0,0.0,4,73429
306377,306377,1547201659806207786,https://logiclike.com/cabinet#/quiz/73429/result,NaN,[],[],NaN,0,1.0,4,73429
306379,306379,1547211249801412828,https://logiclike.com/,NaN,['ip'],['85.141.127.134'],NaN,0,NaN,4,
306381,306381,1547211249801412828,https://logiclike.com/,25230759.0,[],[],NaN,0,NaN,0,


### Вывод по предобработке данных

На данном этапе была проведена предварительная обработка данных:
- данные проверены на пропуски,
- удалены дубликаты,
- названия столбцов приведены к нижнему регистру,
- дабавлены новые столбцы:
 - targetlessons - отметка о возрастной когорте,
 - tarifs - отметка о просмотре тарифов,
 - quiz_status - отметка о решении задачи,
 - new_client - отметка о первом и повторном визите,
 - quiz_number - номер задачи.
 
Вопросы к заказчику, ответы на которые могут внести корректировки в расчеты:
1. Критерии нового пользователя? когда он становится повторным? (в расчетах пользователь с отметкой "Новый пользователь" (установленной при первой последовательности посещения сайта) считался новым пользователем и при последущих последовательностях посещения сайта)
2. Как учитывать пользователей, у которых одновременно несколько возрастных когорт? (в расчетах выбиралась старшая возростная когорта)
3. На какие именно три возрастные когорты требуется разбить пользователей (в данных 5 групп - 0,1,2,3 и пользователи, у которых нет отметки о когорте)? (в работе деление выполнено по 1, 2, 3 когортам)

## Шаг 2. Анализ данных

С помощью сводных таблиц посчитаем:

- Количество пользователей в группе;
- Процент пользователей, достигших цели "Смотрел тарифы";
- Среднее количество уникальных решенных задач на активного пользователя. Активными считаем только тех пользователей, которые просмотрели хотя бы одну задачу. 

In [21]:
df1 = df.pivot_table(index='clientid', aggfunc = ({'targetlessons':'max', 'tarifs':'max', 'quiz_status':'sum', 'new_client':'min'})).reset_index()

In [22]:
df1

,clientid,new_client,quiz_status,targetlessons,tarifs
0,154723112976302,0,18.0,NaN,0
1,1547127081863262,4,1.0,NaN,0
2,1547197740466626,4,0.0,1.0,1
3,1547205546270608,4,0.0,1.0,1
4,1547215394159767,0,25.0,NaN,0
...,...,...,...,...,...
8485,15472391401049904727,0,0.0,NaN,0
8486,15472401471040809633,4,0.0,0.0,1
8487,15472426321068796025,0,0.0,NaN,0
8488,15472769751004263817,0,25.0,1.0,0


In [23]:
df2 = df1.query('targetlessons > 0').pivot_table(index=['targetlessons', 'new_client'], aggfunc = { 'quiz_status':'mean', 'clientid':'count', 'tarifs':'sum'})

In [24]:
df2

clientid  quiz_status  tarifs
targetlessons new_client                               
1.0           0                987    19.040527     304
              4                959    10.868613     516
2.0           0                631    16.234548     147
              4                789     8.934094     400
3.0           0                567    10.171076      72
              4                120     6.883333      59

In [25]:
df2['tarifs'] = df2['tarifs'] / df2['clientid'] * 100

In [26]:
df2.rename(columns = {'clientid':'Количество пользователей', 'tarifs':'Конверсия в просмотр тарифов', 'quiz_status':'Среднее число решенных задач'}, inplace = True)

In [27]:
df2.index.names = ('Возрастная когорта','Тип пользователя')

In [28]:
df2 = df2[['Количество пользователей', 'Конверсия в просмотр тарифов', 'Среднее число решенных задач']]

In [29]:
df3 = df2.T.round(1).rename(columns = {0:'новый', 4:'старый'})

In [30]:
df3

Возрастная когорта              1.0           2.0           3.0       
Тип пользователя              новый старый  новый старый  новый старый
Количество пользователей      987.0  959.0  631.0  789.0  567.0  120.0
Конверсия в просмотр тарифов   30.8   53.8   23.3   50.7   12.7   49.2
Среднее число решенных задач   19.0   10.9   16.2    8.9   10.2    6.9

### Вывод по анализу данных

На основании полученных данных можно сделать вывод о том, что 1 возрастная когорта пользователей является наиболее выгодной для привлечения, т.к.:
- выше конверсия в просмотр тарифов,
- больше среднее число решенных задач,
- является самой крупной по количеству пользователей.

## Общий вывод

По заданию необходимо сделать вывод о том, какая возрастная когорта пользователей является наиболее выгодной для привлечения.

В данной работе был проведен следующий анализ:
1. Предварительная обработка данных:
- данные проверены на пропуски,
- удалены дубликаты,
- названия столбцов приведены к нижнему регистру,
- дабавлены новые столбцы:
 - targetlessons - отметка о возрастной когорте,
 - tarifs - отметка о просмотре тарифов,
 - quiz_status - отметка о решении задачи,
 - new_client - отметка о первом и повторном визите,
 - quiz_number - номер задачи.
2. Группировка и анализ данных:

In [31]:
df3

Возрастная когорта              1.0           2.0           3.0       
Тип пользователя              новый старый  новый старый  новый старый
Количество пользователей      987.0  959.0  631.0  789.0  567.0  120.0
Конверсия в просмотр тарифов   30.8   53.8   23.3   50.7   12.7   49.2
Среднее число решенных задач   19.0   10.9   16.2    8.9   10.2    6.9

На основании полученных данных можно сделать вывод о том, что 1 возрастная когорта пользователей является наиболее выгодной для привлечения, т.к.:
- выше конверсия в просмотр тарифов,
- больше среднее число решенных задач,
- является самой крупной по количеству пользователей.

Вопросы к заказчику, ответы на которые могут внести корректировки в расчеты:
1. Критерии нового пользователя? когда он становится повторным? (в расчетах пользователь с отметкой "Новый пользователь" (установленной при первой последовательности посещения сайта) считался новым пользователем и при последущих последовательностях посещения сайта)
2. Как учитывать пользователей, у которых одновременно несколько возрастных когорт? (в расчетах выбиралась старшая возростная когорта)
3. На какие именно три возрастные когорты требуется разбить пользователей (в данных 5 групп - 0,1,2,3 и пользователи, у которых нет отметки о когорте)? (в работе деление выполнено по 1, 2, 3 когортам)